In [1]:
import pandas as pd
import numpy as np
import requests

In [40]:
# Read death data from JHU
us_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

world_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')

In [41]:
START_CASES = 10
END_CASES = 500

In [42]:
# filter out areas with fewer than 1000 deaths now
us_deaths = us_deaths.groupby(by='Province_State').agg('sum')
us_deaths = us_deaths[us_deaths[us_deaths.columns[-1]]>=END_CASES]
world_deaths = world_deaths[world_deaths[world_deaths.columns[-1]]>=END_CASES]

In [43]:
# get days from START_CASES to END_CASES
start_date = pd.to_datetime('01-22-2020')
world_end_date = world_deaths.columns[-1]
us_end_date = us_deaths.columns[-1]

In [44]:
# get us_starts and us_ends
us_starts = {}
us_ends = {}

for d in pd.date_range(start_date, us_end_date):
    date = f'{d.month}/{d.day}/{str(d.year)[-2:]}'
    for state in us_deaths.index:
        deaths = us_deaths[date][state]
        if state not in us_starts and deaths>=START_CASES:
            us_starts[state] = date
        if state not in us_ends and deaths>=END_CASES:
            us_ends[state] = date

In [45]:
# get world_starts and world_ends
world_starts = {}
world_ends = {}

for d in pd.date_range(start_date, world_end_date):
    date = f'{d.month}/{d.day}/{str(d.year)[-2:]}'
    for country in world_deaths.index:
        deaths = world_deaths[date][country]
        if country not in world_starts and deaths>=START_CASES:
            world_starts[country] = date
        if country not in world_ends and deaths>=END_CASES:
            world_ends[country] = date

In [46]:
# create series of start_dates and end_dates
us_start_dates = pd.Series(us_starts)
us_start_dates.name = 'start_date'
us_end_dates = pd.Series(us_ends)
us_end_dates.name = 'end_date'

world_start_dates = pd.Series(world_starts)
world_start_dates.name = 'start_date'
world_end_dates = pd.Series(world_ends)
world_end_dates.name = 'end_date'


In [47]:
world_deaths.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20
16,NaN,Austria,47.5162,14.5501,0,0,0,0,0,0,...,384,393,410,431,443,452,470,491,510,522
23,NaN,Belgium,50.8333,4.0000,0,0,0,0,0,0,...,4157,4440,4857,5163,5453,5683,5828,5998,6262,6490
28,NaN,Brazil,-14.2350,-51.9253,0,0,0,0,0,0,...,1532,1736,1924,2141,2354,2462,2587,2741,2906,3331
42,Ontario,Canada,51.2538,-85.3232,0,0,0,0,0,0,...,334,385,490,524,564,591,624,694,762,806
44,Quebec,Canada,52.9399,-73.5491,0,0,0,0,0,0,...,435,487,630,688,688,820,939,1044,1134,1243


In [48]:
# add start and end date columns
us_deaths = pd.merge(us_deaths, us_start_dates, left_index=True, right_index=True)
us_deaths = pd.merge(us_deaths, us_end_dates, left_index=True, right_index=True)

world_deaths = pd.merge(world_deaths, world_start_dates, left_index=True, right_index=True)
world_deaths = pd.merge(world_deaths, world_end_dates, left_index=True, right_index=True)

In [49]:
# getting region name in Region column
def get_region(row):
    if row['Province/State'] is not np.NaN:
        row['Region'] = row['Province/State']+', '+row['Country/Region']
    else:
        row['Region'] = row['Country/Region']
    return row

world_deaths = world_deaths.apply(get_region, axis=1)
us_deaths['Region'] = us_deaths.index

In [50]:
us_drop_cols = ['UID', 'code3', 'FIPS', 'Lat', 'Long_', 'Population']
world_drop_cols = ['Lat', 'Long']
us_deaths.drop(us_drop_cols, axis='columns', inplace=True)
world_deaths.drop(world_drop_cols, axis='columns', inplace=True)

In [83]:
if world_deaths['Country/Region'].str.contains(world_deaths['Country/Region'].iloc[1]).any():
    print('True')
else:
    print('False')

True


In [6]:
# Read country population data, data from here: https://www.worldometers.info/world-population/population-by-country/

populations = pd.read_csv('country_populations.csv')

In [13]:
pop_density = pd.read_csv('population_density.csv')

In [61]:
### Getting weather data ###

# setup for weather api rewquests
# TODO put this in env file or something


headers = {
    'token': token,
}

In [58]:
# test = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/datasets?location_id=FIPS:01', headers=headers)
# test = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=FIPS:01&startdate=2010-05-01&enddate=2010-05-02&units=standard&datatypeid=TMAX', headers=headers)
# test = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&locationid=ZIP:28801&startdate=2010-05-01&enddate=2010-05-01&units=standard', headers=headers)
locations = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CNTRY&limit=1000', headers=headers)
location_categories = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/locationcategories', headers=headers)


In [64]:
# get state location codes
state_res = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=ST&limit=52', headers=headers)

state_codes = {}
for s in state_res.json()['results']:
    if s['name'] in us_deaths['Region']:
        state_codes[s['name']] = s['id']

In [84]:
# get country location codes
country_res = requests.get('https://www.ncdc.noaa.gov/cdo-web/api/v2/locations?locationcategoryid=CNTRY&limit=1000', headers=headers)

country_codes = {}
for c in country_res.json()['results']:
    print(c)
    if world_deaths['Country/Region'].str.contains(c['name']).any():
        country_codes[c['name']] = s['id']

'name': 'Belgium', 'datacoverage': 0.9993, 'id': 'FIPS:BE'}
{'mindate': '1951-01-01', 'maxdate': '2020-04-22', 'name': 'The Bahamas', 'datacoverage': 1, 'id': 'FIPS:BF'}
{'mindate': '1901-01-01', 'maxdate': '2020-04-18', 'name': 'Bangladesh', 'datacoverage': 1, 'id': 'FIPS:BG'}
{'mindate': '1960-11-01', 'maxdate': '2020-04-18', 'name': 'Belize', 'datacoverage': 0.7263, 'id': 'FIPS:BH'}
{'mindate': '1897-10-01', 'maxdate': '2020-04-18', 'name': 'Bosnia & Herzegovina', 'datacoverage': 1, 'id': 'FIPS:BK'}
{'mindate': '1942-06-01', 'maxdate': '2020-04-18', 'name': 'Bolivia', 'datacoverage': 1, 'id': 'FIPS:BL'}
{'mindate': '1944-07-13', 'maxdate': '2020-04-18', 'name': 'Myanmar', 'datacoverage': 1, 'id': 'FIPS:BM'}
{'mindate': '1941-01-01', 'maxdate': '2020-04-18', 'name': 'Benin', 'datacoverage': 1, 'id': 'FIPS:BN'}
{'mindate': '1875-07-01', 'maxdate': '2020-04-18', 'name': 'Belarus', 'datacoverage': 1, 'id': 'FIPS:BO'}
{'mindate': '1943-03-16', 'maxdate': '1949-10-31', 'name': 'Solomon Is

{'Austria': 'FIPS:56',
 'Belgium': 'FIPS:56',
 'Brazil': 'FIPS:56',
 'Canada': 'FIPS:56',
 'China': 'FIPS:56',
 'Ecuador': 'FIPS:56',
 'Ireland': 'FIPS:56',
 'France': 'FIPS:56',
 'Germany': 'FIPS:56',
 'Indonesia': 'FIPS:56',
 'India': 'FIPS:56',
 'Iran': 'FIPS:56',
 'Italy': 'FIPS:56',
 'Mexico': 'FIPS:56',
 'Netherlands': 'FIPS:56',
 'Peru': 'FIPS:56',
 'Portugal': 'FIPS:56',
 'Romania': 'FIPS:56',
 'Russia': 'FIPS:56',
 'Spain': 'FIPS:56',
 'Sweden': 'FIPS:56',
 'Switzerland': 'FIPS:56',
 'Turkey': 'FIPS:56',
 'United Kingdom': 'FIPS:56'}

In [69]:
world_deaths.head(20)

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,start_date,end_date,Region
16,NaN,Austria,0,0,0,0,0,0,0,0,...,431,443,452,470,491,510,522,3/22/20,4/22/20,Austria
23,NaN,Belgium,0,0,0,0,0,0,0,0,...,5163,5453,5683,5828,5998,6262,6490,3/17/20,3/30/20,Belgium
28,NaN,Brazil,0,0,0,0,0,0,0,0,...,2141,2354,2462,2587,2741,2906,3331,3/20/20,4/6/20,Brazil
42,Ontario,Canada,0,0,0,0,0,0,0,0,...,524,564,591,624,694,762,806,3/26/20,4/17/20,"Ontario, Canada"
44,Quebec,Canada,0,0,0,0,0,0,0,0,...,688,688,820,939,1044,1134,1243,3/27/20,4/16/20,"Quebec, Canada"
62,Hubei,China,17,17,24,40,52,76,125,125,...,4512,4512,4512,4512,4512,4512,4512,1/22/20,2/5/20,"Hubei, China"
97,NaN,Ecuador,0,0,0,0,0,0,0,0,...,421,456,474,507,520,537,560,3/22/20,4/20/20,Ecuador
116,NaN,France,0,0,0,0,0,0,0,0,...,18681,19323,19718,20265,20796,21340,21856,3/7/20,3/21/20,France
120,NaN,Germany,0,0,0,0,0,0,0,0,...,4352,4459,4586,4862,5033,5279,5575,3/15/20,3/29/20,Germany
131,NaN,India,0,0,0,0,0,0,0,0,...,486,521,559,592,645,681,721,3/23/20,4/18/20,India


In [67]:
country_res.json()

'mindate': '1966-02-01',
   'maxdate': '2020-04-22',
   'name': 'Cayman Is.',
   'datacoverage': 1,
   'id': 'FIPS:CJ'},
  {'mindate': '1948-01-01',
   'maxdate': '2020-04-18',
   'name': 'Cameroon',
   'datacoverage': 1,
   'id': 'FIPS:CM'},
  {'mindate': '1941-03-02',
   'maxdate': '2020-04-18',
   'name': 'Colombia',
   'datacoverage': 1,
   'id': 'FIPS:CO'},
  {'mindate': '1955-07-04',
   'maxdate': '2020-04-18',
   'name': 'Costa Rica',
   'datacoverage': 1,
   'id': 'FIPS:CS'},
  {'mindate': '1950-01-01',
   'maxdate': '2020-04-18',
   'name': 'Central African Republic',
   'datacoverage': 1,
   'id': 'FIPS:CT'},
  {'mindate': '1942-07-01',
   'maxdate': '2020-04-23',
   'name': 'Cuba',
   'datacoverage': 1,
   'id': 'FIPS:CU'},
  {'mindate': '1973-01-01',
   'maxdate': '2020-04-18',
   'name': 'Cape Verde',
   'datacoverage': 0.9819,
   'id': 'FIPS:CV'},
  {'mindate': '1973-01-01',
   'maxdate': '2020-04-18',
   'name': 'Cook Is.',
   'datacoverage': 1,
   'id': 'FIPS:CW'},
  {'

In [65]:
state_codes

{'California': 'FIPS:06',
 'Colorado': 'FIPS:08',
 'Connecticut': 'FIPS:09',
 'Florida': 'FIPS:12',
 'Georgia': 'FIPS:13',
 'Illinois': 'FIPS:17',
 'Indiana': 'FIPS:18',
 'Louisiana': 'FIPS:22',
 'Maryland': 'FIPS:24',
 'Massachusetts': 'FIPS:25',
 'Michigan': 'FIPS:26',
 'New Jersey': 'FIPS:34',
 'New York': 'FIPS:36',
 'Ohio': 'FIPS:39',
 'Pennsylvania': 'FIPS:42',
 'Texas': 'FIPS:48',
 'Washington': 'FIPS:53'}